In [25]:
import numpy as np
import pandas as pd
import os

def get_run(seed,experiment,root):
    log = pd.read_csv(os.path.join(root,str(seed),experiment,"log.csv"))
    test = log.loc[log["mode"]=="test"]
    return test.iloc[-1]#.sort_values(by="kappa", ascending=False).iloc[0]

def table_row(means, stds,sigmainterval=1):
    line = list()
    for mean, std in zip(means, stds):
        line.append(f"{mean:.2f}" + "$^{\pm" + f"{std*sigmainterval:.2f}" + "}$")
    return " & ".join(line)

def get_acc_kappa(root):

    isprs_tum_rnn = [get_run(seed,"isprs_tum_rnn",root) for seed in [0,1,2]]
    isprs_gaf_rnn = [get_run(seed,"isprs_gaf_rnn",root) for seed in [0,1,2]]
    isprs_tum_transformer = [get_run(seed,"isprs_tum_transformer",root) for seed in [0,1,2]]
    isprs_gaf_transformer = [get_run(seed,"isprs_gaf_transformer",root) for seed in [0,1,2]]
    isprs_tum_msresnet = [get_run(seed,"isprs_tum_msresnet",root) for seed in [0,1,2]]
    isprs_gaf_msresnet = [get_run(seed,"isprs_gaf_msresnet",root) for seed in [0,1,2]]
    isprs_tum_tempcnn = [get_run(seed,"isprs_tum_tempcnn",root) for seed in [0,1,2]]
    isprs_gaf_tempcnn = [get_run(seed,"isprs_gaf_tempcnn",root) for seed in [0,1,2]]

    isprs_tum_rnn_kappa = np.array([run.kappa for run in isprs_tum_rnn])
    isprs_gaf_rnn_kappa = np.array([run.kappa for run in isprs_gaf_rnn])
    isprs_tum_transformer_kappa = np.array([run.kappa for run in isprs_tum_transformer])
    isprs_gaf_transformer_kappa = np.array([run.kappa for run in isprs_gaf_transformer])
    isprs_tum_msresnet_kappa = np.array([run.kappa for run in isprs_tum_msresnet])
    isprs_gaf_msresnet_kappa = np.array([run.kappa for run in isprs_gaf_msresnet])
    isprs_tum_tempcnn_kappa = np.array([run.kappa for run in isprs_tum_tempcnn])
    isprs_gaf_tempcnn_kappa = np.array([run.kappa for run in isprs_gaf_tempcnn])

    isprs_tum_rnn_accuracy = np.array([run.accuracy for run in isprs_tum_rnn])
    isprs_gaf_rnn_accuracy = np.array([run.accuracy for run in isprs_gaf_rnn])
    isprs_tum_transformer_accuracy = np.array([run.accuracy for run in isprs_tum_transformer])
    isprs_gaf_transformer_accuracy = np.array([run.accuracy for run in isprs_gaf_transformer])
    isprs_tum_msresnet_accuracy = np.array([run.accuracy for run in isprs_tum_msresnet])
    isprs_gaf_msresnet_accuracy = np.array([run.accuracy for run in isprs_gaf_msresnet])
    isprs_tum_tempcnn_accuracy = np.array([run.accuracy for run in isprs_tum_tempcnn])
    isprs_gaf_tempcnn_accuracy = np.array([run.accuracy for run in isprs_gaf_tempcnn])

    tum_kappa = [isprs_tum_rnn_kappa.mean(),isprs_tum_transformer_kappa.mean(),isprs_tum_msresnet_kappa.mean(),isprs_tum_tempcnn_kappa.mean()]
    tum_kappa_std = [isprs_tum_rnn_kappa.std(),isprs_tum_transformer_kappa.std(),isprs_tum_msresnet_kappa.std(),isprs_tum_tempcnn_kappa.std()]

    gaf_kappa = [isprs_gaf_rnn_kappa.mean(),isprs_gaf_transformer_kappa.mean(),isprs_gaf_msresnet_kappa.mean(),isprs_gaf_tempcnn_kappa.mean()]
    gaf_kappa_std = [isprs_gaf_rnn_kappa.std(),isprs_gaf_transformer_kappa.std(),isprs_gaf_msresnet_kappa.std(),isprs_gaf_tempcnn_kappa.std()]

    tum_acc = [isprs_tum_rnn_accuracy.mean(),isprs_tum_transformer_accuracy.mean(),isprs_tum_msresnet_accuracy.mean(),isprs_tum_tempcnn_accuracy.mean()]
    tum_acc_std = [isprs_tum_rnn_accuracy.std(),isprs_tum_transformer_accuracy.std(),isprs_tum_msresnet_accuracy.std(),isprs_tum_tempcnn_accuracy.std()]

    gaf_acc = [isprs_gaf_rnn_accuracy.mean(),isprs_gaf_transformer_accuracy.mean(),isprs_gaf_msresnet_accuracy.mean(),isprs_gaf_tempcnn_accuracy.mean()]
    gaf_acc_std = [isprs_gaf_rnn_accuracy.std(),isprs_gaf_transformer_accuracy.std(),isprs_gaf_msresnet_accuracy.std(),isprs_gaf_tempcnn_accuracy.std()]

    return tum_kappa, tum_kappa_std, gaf_kappa, gaf_kappa_std, tum_acc, tum_acc_std, gaf_acc, gaf_acc_std
    
def fill_table(rows,desc="ov.acc"):
    return r"""\begin{tabular}{lcccc}
\toprule
"""+desc+r""" & \rnn & \transformer & \msresnet & \tempcnn \\
\cmidrule(lr){2-2}\cmidrule(lr){3-3}\cmidrule(lr){4-4}\cmidrule(lr){5-5}
    """+rows+r"""
\bottomrule
\end{tabular}"""

def make_table(exp1,exp1_std,exp2,exp2_std, desc):
    
    line1 = f"\tprocessed & "+table_row(exp1,exp1_std,sigmainterval=1) + " \\\ "
    line2 = f"\traw       & "+table_row(exp2,exp2_std,sigmainterval=1) + " \\\ "
    rows = "\n".join([line1,line2])
    
    return fill_table(rows,desc)

In [26]:
tableroot = "/home/marc/projects/ISPRS_CropTypeMapping/tables/preraw"
root = "/data/isprs/preraw.isprs"

tum_kappa, tum_kappa_std, gaf_kappa, gaf_kappa_std, tum_acc, tum_acc_std, gaf_acc, gaf_acc_std = get_acc_kappa(root)

print("kappa")
print()
table=make_table(gaf_kappa,gaf_kappa_std,tum_kappa,tum_kappa_std, "kappa")
print(table)
print()
fname = os.path.join(tableroot,"kappa.tex")
print(f"writing to {fname}")
print(table,file=open(fname,'w'))
print()
print("accuracy")
print()
table=make_table(gaf_acc,gaf_acc_std,tum_acc,tum_acc_std, "accuracy")
print(table)
print()
fname = os.path.join(tableroot,"accuracy.tex")
print(f"writing to {fname}")
print(table,
      file=open(fname,'w'))
print()

kappa

\begin{tabular}{lcccc}
\toprule
kappa & \rnn & \transformer & \msresnet & \tempcnn \\
\cmidrule(lr){2-2}\cmidrule(lr){3-3}\cmidrule(lr){4-4}\cmidrule(lr){5-5}
    	processed & 0.78$^{\pm0.01}$ & 0.79$^{\pm0.02}$ & 0.76$^{\pm0.02}$ & 0.80$^{\pm0.00}$ \\ 
	raw       & 0.71$^{\pm0.01}$ & 0.71$^{\pm0.02}$ & 0.69$^{\pm0.02}$ & 0.67$^{\pm0.01}$ \\ 
\bottomrule
\end{tabular}

writing to /home/marc/projects/ISPRS_CropTypeMapping/tables/preraw/kappa.tex

accuracy

\begin{tabular}{lcccc}
\toprule
accuracy & \rnn & \transformer & \msresnet & \tempcnn \\
\cmidrule(lr){2-2}\cmidrule(lr){3-3}\cmidrule(lr){4-4}\cmidrule(lr){5-5}
    	processed & 0.85$^{\pm0.01}$ & 0.85$^{\pm0.02}$ & 0.83$^{\pm0.02}$ & 0.86$^{\pm0.00}$ \\ 
	raw       & 0.81$^{\pm0.01}$ & 0.80$^{\pm0.02}$ & 0.79$^{\pm0.02}$ & 0.79$^{\pm0.00}$ \\ 
\bottomrule
\end{tabular}

writing to /home/marc/projects/ISPRS_CropTypeMapping/tables/preraw/accuracy.tex



In [27]:
[get_run(seed,"isprs_tum_rnn",root) for seed in [0,1,2]]

[Unnamed: 0             222
 epoch                  112
 iteration              NaN
 mode                  test
 accuracy           0.81545
 loss              0.662889
 mean_accuracy     0.417018
 mean_recall       0.417018
 mean_precision    0.533647
 mean_f1           0.442969
 kappa             0.719655
 Name: 223, dtype: object, Unnamed: 0             148
 epoch                   76
 iteration              NaN
 mode                  test
 accuracy          0.805772
 loss              0.774324
 mean_accuracy     0.402454
 mean_recall       0.402454
 mean_precision     0.49386
 mean_f1            0.42627
 kappa             0.710515
 Name: 151, dtype: object, Unnamed: 0             174
 epoch                   89
 iteration              NaN
 mode                  test
 accuracy          0.797057
 loss              0.747421
 mean_accuracy      0.43348
 mean_recall        0.43348
 mean_precision    0.481631
 mean_f1           0.432808
 kappa             0.703148
 Name: 177, dtype: objec

In [28]:
tableroot = "/home/marc/projects/ISPRS_CropTypeMapping/tables/preraw2"
root = "/data/isprs/preraw2"

tum_kappa, tum_kappa_std, gaf_kappa, gaf_kappa_std, tum_acc, tum_acc_std, gaf_acc, gaf_acc_std = get_acc_kappa(root)

print("kappa")
print()
table=make_table(gaf_kappa,gaf_kappa_std,tum_kappa,tum_kappa_std, "kappa")
print(table)
print()
fname = os.path.join(tableroot,"kappa.tex")
print(f"writing to {fname}")
print(table,file=open(fname,'w'))
print()
print("accuracy")
print()
table=make_table(gaf_acc,gaf_acc_std,tum_acc,tum_acc_std, "accuracy")
print(table)
print()
fname = os.path.join(tableroot,"accuracy.tex")
print(f"writing to {fname}")
print(table,
      file=open(fname,'w'))
print()

kappa

\begin{tabular}{lcccc}
\toprule
kappa & \rnn & \transformer & \msresnet & \tempcnn \\
\cmidrule(lr){2-2}\cmidrule(lr){3-3}\cmidrule(lr){4-4}\cmidrule(lr){5-5}
    	processed & 0.87$^{\pm0.01}$ & 0.87$^{\pm0.03}$ & 0.85$^{\pm0.01}$ & 0.88$^{\pm0.02}$ \\ 
	raw       & 0.83$^{\pm0.00}$ & 0.82$^{\pm0.02}$ & 0.78$^{\pm0.02}$ & 0.71$^{\pm0.05}$ \\ 
\bottomrule
\end{tabular}

writing to /home/marc/projects/ISPRS_CropTypeMapping/tables/preraw2/kappa.tex

accuracy

\begin{tabular}{lcccc}
\toprule
accuracy & \rnn & \transformer & \msresnet & \tempcnn \\
\cmidrule(lr){2-2}\cmidrule(lr){3-3}\cmidrule(lr){4-4}\cmidrule(lr){5-5}
    	processed & 0.92$^{\pm0.01}$ & 0.92$^{\pm0.02}$ & 0.91$^{\pm0.01}$ & 0.92$^{\pm0.01}$ \\ 
	raw       & 0.90$^{\pm0.00}$ & 0.89$^{\pm0.01}$ & 0.87$^{\pm0.01}$ & 0.83$^{\pm0.04}$ \\ 
\bottomrule
\end{tabular}

writing to /home/marc/projects/ISPRS_CropTypeMapping/tables/preraw2/accuracy.tex

